In [9]:
import os

In [10]:
%pwd

'c:\\Users\\Jayant Dadhich\\Jupyter\\01KrishNaik\\Practice\\03_DL_Project\\E2E_Kidney_Disease_Classification'

In [3]:
os.chdir("../")

In [4]:
pwd

'c:\\Users\\Jayant Dadhich\\Jupyter\\01KrishNaik\\Practice\\03_DL_Project\\E2E_Kidney_Disease_Classification'

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)         # if frozen=True, user can't add anything here
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weight: str
    params_classes: int


In [13]:
os.chdir("c:\\Users\\Jayant Dadhich\\Jupyter\\01KrishNaik\\Practice\\03_DL_Project\\E2E_Kidney_Disease_Classification")

In [18]:
from src.cnnClassifier.constants import *


In [19]:

from src.cnnClassifier.utils.common import read_yaml,create_directories

ModuleNotFoundError: No module named 'cnnClassifier'

In [21]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) ->  PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([config.root_dir])
        
        prepare_base_model_config = PrepareBaseModelConfig(
        root_dir = Path(config.root_dir),
        base_model_path=Path(config.base_model_path),
        updated_base_model_path=Path(config.updated_base_model_path),
        params_image_size=self.params.IMAGE_SIZE,
        params_learning_rate=self.params.LEARNING_RATE,
        params_include_top=self.params.INCLUDE_TOP,
        params_weights=self.params.WEIGHTS,
        params_classes=self.params.CLASSES
        )
        return prepare_base_model_config
    
    


In [24]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [23]:
pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/fb/59/3eb58629e3749d9f4fc1e522487af369f9bd4c451f465d3054961fab6bf8/tensorflow-2.13.1-cp38-cp38-win_amd64.whl.metadata
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/5b/6f/8b125d126d54061c0be610b135aaf2f8960f55c2e185ac32445e4a5012d5/tensorflow-2.13.0-cp38-cp38-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.13.0 from https://files.pythonhosted.org/packages/38/ba/dd4d998a852451e98dc009ecb208bbb0eeb0c8252dc35b7c4e1050762b36/tensorflow_intel-2.13.0-cp38-cp38-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.w

In [25]:
class PrepareBaseModel:
    def __init__(self, config:PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.application.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weight,
            include_top=self.config.params_include_top
        )
        self.save_model(path=self.config.base_model_path, model=self.model)
        
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)
    
        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [26]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()

except Exception as e:
    raise e


NameError: name 'read_yaml' is not defined